In [1]:
# Data Enrichment and Neo4j Loading

import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Example usage:
year = 1965
side = 'west'

enricher = DataEnricher(year, side)
enricher.load_and_combine_data()
enricher.add_administrative_data()
enricher.add_east_west_classification()
enricher.enrich_transport_data()
final_df = enricher.prepare_for_neo4j()

# Save enriched data
output_path = f'../data/processed/stops_{year}_{side}_final.csv'
final_df.to_csv(output_path, index=False)
logger.info(f"Saved enriched data to {output_path}")